In [1]:
import grape
import networkx as nx
import obonet
import os
import pandas as pd

2024-01-18 14:18:11.283976: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
INFO:pykeen.utils:Using opt_einsum


In [2]:
lotus = pd.read_csv("./data/molecules/230106_frozen_metadata.csv.gz", low_memory=False)

In [3]:
lotus['wd_taxon'] = lotus['organism_wikidata'].str.extract(r"(Q\d+)")
lotus['wd_taxon'] = 'wd:' + lotus['wd_taxon']

# Prepare species phylogeny

We are first going to get from wikidata all the species that are present in lotus. To do so we use the self made function to run it. If the file already exists in our system just load the file and don't run the function. Here the species will be taken from wikidata and will have their corresponding wikidata url as identifier. 

In [4]:
from wikidata import taxonomy_in_edges
from tqdm import tqdm
tqdm.pandas()

In [5]:
if os.path.isfile("./data/species/species_edges.csv"):
    species_phylo = pd.read_csv("./data/species/species_edges.csv", index_col=0)
else:
    res = lotus.wd_taxon.drop_duplicates().reset_index(drop=True).progress_apply(taxonomy_in_edges)
    species_phylo = pd.concat(list(res)).drop_duplicates().dropna().reset_index(drop=True)
    species_phylo.rename(columns={0:"child", 1:"parent"}, inplace=True)
    species_phylo['child'] = 'wd:' + species_phylo['child'].str.extract(r"(Q\d+)")
    species_phylo['parent'] = 'wd:' + species_phylo['parent'].str.extract(r"(Q\d+)")
    species_phylo['type'] = 'biolink:subclass_of'
    species_phylo.to_csv("./data/species/species_edges.csv")

In [7]:
species_nodes = pd.DataFrame({'node': pd.concat([species_phylo.child, species_phylo.parent]).drop_duplicates().values,
                              'type': 'biolink:OrganismTaxon'})

In [10]:
species_nodes.to_csv("./data/species/species_nodes.csv")